In [1]:
import os
import re
import pymorphy2
import numpy as np

In [2]:
from sklearn.naive_bayes import MultinomialNB

# Load Data

In [3]:
politics = './politics/'
sports = './sports/'
unclassified = './unclassified/'

In [4]:
classes = np.array([u'Politics', u'Sports'])

In [5]:
train_docs = []
train_labels = []
test_docs = []

In [6]:
def load_train(cat, label):
    global train_docs, train_labels
    for filename in os.listdir(cat):
        if filename.startswith('news'):
            with open(cat+filename, 'r') as f:
                train_docs.append(f.read().decode('utf-8'))
                train_labels.append(label)

In [7]:
def load_test(cat):
    global test_docs
    for filename in os.listdir(cat):
        if filename.startswith('news'):
            with open(cat+filename, 'r') as f:
                test_docs.append(f.read().decode('utf-8'))

In [8]:
load_train(politics, 0)

In [9]:
load_train(sports, 1)

In [10]:
load_test(unclassified)

In [11]:
names = [filename for filename in os.listdir(unclassified) if filename.startswith('news')]
names

['news_antifashisty',
 'news_britansky_ministr',
 'news_dic_advocat',
 'news_gol_azmuna',
 'news_kirgiziya',
 'news_lavrov_o_prosbe_kieva',
 'news_net_otsrochki',
 'news_promes_dumayu',
 'news_savchenko']

In [12]:
test_labels = [0, 0, 1, 1, 0, 0, 0, 1, 0]

# Preprocess Data

In [13]:
words_splitter = re.compile(r'\w+', re.U)

In [14]:
cache = {}

In [15]:
def tokenizer(text):
    global cache
    lemmatizer = pymorphy2.MorphAnalyzer()
    result = []
    for word in words_splitter.findall(text):
        word_hash = hash(word)
        if word_hash not in cache:
            cache[word_hash] = lemmatizer.parse(word)[0].normal_form            
        result.append(cache[word_hash])
    return result

# Vectorizer

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
vectorizer = TfidfVectorizer(tokenizer=tokenizer)

In [18]:
X_train = vectorizer.fit_transform(train_docs).toarray()

In [19]:
X_test = vectorizer.transform(test_docs).toarray()

# Classifier

In [20]:
mnb_clf = MultinomialNB(alpha = 0.1)

In [21]:
def predict_proba(X_test, with_log = True):
    global mnb_clf
    if with_log:
        predicted = mnb_clf.predict_log_proba(X_test)
    else:
        predicted = mnb_clf.predict_proba(X_test)
    return predicted

In [22]:
def print_predicted(n, predicted, text):
    global names, classes, test_docs
    print u'Weights for class "{0}": {1}'.format(classes[0], predicted[n][0])
    print u'Weights for class "{0}": {1}'.format(classes[1], predicted[n][1])
    print
    print u'Predicted class for text: {0}\n{1}\n{2}...'.format(classes[np.argmax(predicted[n])],
                                                              u'-'*40, text[:500])

In [23]:
def classify_text(text):
    global vectorizer
    X = vectorizer.transform([text])
    predicted = predict_proba(X)
    print_predicted(0, predicted, text)

In [24]:
mnb_clf.fit(X_train, train_labels)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

# Test Text

In [25]:
classify_text(u'президент России')

Weights for class "Politics": -0.236280304978
Weights for class "Sports": -1.55855149695

Predicted class for text: Politics
----------------------------------------
президент России...


In [26]:
classify_text(u'доклад по теме')

Weights for class "Politics": -0.447018487884
Weights for class "Sports": -1.02035232641

Predicted class for text: Politics
----------------------------------------
доклад по теме...


In [27]:
classify_text(u'футбольная команда')

Weights for class "Politics": -2.00041637266
Weights for class "Sports": -0.14534830389

Predicted class for text: Sports
----------------------------------------
футбольная команда...


In [28]:
classify_text(u'игровое поле')

Weights for class "Politics": -0.880523807043
Weights for class "Sports": -0.535401318063

Predicted class for text: Sports
----------------------------------------
игровое поле...


# Test Data

In [29]:
print_predicted(1, predict_proba(X_test), test_docs[1])

Weights for class "Politics": -0.125107774807
Weights for class "Sports": -2.14048152234

Predicted class for text: Politics
----------------------------------------
Британский министр иностранных дел прокомментировал годовщину воссоединения Крыма с Россией. По его словам, полуостров должен стать частью Украины. 

Великобритания
Единая Россия
Крым
Филип Хаммонд, глава МИД Великобритании: «Наше послание России ясно — присоединение Крыма в марте 2014 года было незаконным, остается оно незаконным и в марте 2015 года».

По его словам, такое поведение России угрожает международной безопасности. Филип Хаммонд подчеркнул, что Британия не признает прошлогодний рефер...


In [30]:
print_predicted(2, predict_proba(X_test), test_docs[2])

Weights for class "Politics": -1.96384033471
Weights for class "Sports": -0.151193324396

Predicted class for text: Sports
----------------------------------------
ДИК АДВОКАТ: "САНДЕРЛЕНД" ЗАСЛУЖИВАЛ БОЛЬШЕГО"

Главный тренер "Сандерленда" Дик Адвокат после поражения от "Вест Хэма" (0:1) в 30-м туре чемпионата Англии остался недоволен результатом, однако отметил, что игра указала на недостатки. По словам голландца, если их исправить, то в следующем матче против "Ньюкасла" можно добиться победы. Напомним, для бывшего наставника "Зенита" прошедшая игра стала дебютной на посту главного тренера Сандерленда.

– Мы заслуживали большего, – цитирует Адвоката BBC....


In [31]:
print_predicted(3, predict_proba(X_test), test_docs[3])

Weights for class "Politics": -2.97835945911
Weights for class "Sports": -0.0522160673529

Predicted class for text: Sports
----------------------------------------
Гол Азмуна принес "Ростову" победу над "Кубанью" со счетом 2:1.

В Ростове-на-Дону на стадионе "Олимп-2" состоялся заключительный матч 19 тура чемпионата России по футболу. Аутсайдер премьер-лиги ФК "Ростов" принимал на своем поле "Кубань" из Краснодара, которая на начало матча находилась на восьмом месте турнирной таблицы. Встреча закончилась со счетом 2:1. Все мячи были забиты во втором тайме.

Отметим, что еще задолго до встречи болельщики и сами игроки назвали встречу "Кубани" и "Ростова" - ...


In [32]:
print_predicted(4, predict_proba(X_test), test_docs[4])

Weights for class "Politics": -0.145483241225
Weights for class "Sports": -1.99955426514

Predicted class for text: Politics
----------------------------------------
Киргизия намерена вступить в ЕАЭС до 9 мая.

В Петербурге в понедельник прошли переговоры президентов России и Киргизии Владимира Путина и Алмазбека Атамбаева. В числе прочего, обсуждался процесс вхождения Киргизии в Евразийский экономический союз. По мнению российского лидера, чем быстрее будет выполнена эта задача, тем лучше.

Планы укрепления сотрудничества России и Киргизии обсудили в понедельник президенты двух стран. Переговоры Владимира Путина и Алмазбека Атамбаева прошли в Константиновск...
